In [1]:
# proj1
# https://drive.google.com/file/d/12aWDY1meozhuUu1Ci2uzj9ZVjI_yTxJ-/view?usp=share_link
!gdown --id 12aWDY1meozhuUu1Ci2uzj9ZVjI_yTxJ-
!unzip proj1.zip

# watermark1
# https://drive.google.com/file/d/1kdxK8ynt1_UNen3UcoFd4OFIhfPDGQaQ/view?usp=share_link
!gdown --id 1kdxK8ynt1_UNen3UcoFd4OFIhfPDGQaQ
!unzip watermark1.zip

/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=12aWDY1meozhuUu1Ci2uzj9ZVjI_yTxJ-
To: /content/proj1.zip
100% 413k/413k [00:00<00:00, 33.9MB/s]
Archive:  proj1.zip
   creating: proj1/
  inflating: __MACOSX/._proj1        
  inflating: proj1/.DS_Store         
  inflating: __MACOSX/proj1/._.DS_Store  
   creating: proj1/proj1/
  inflating: proj1/proj1/.DS_Store   
  inflating: __MACOSX/proj1/proj1/._.DS_Store  
  inflating: proj1/proj1/result.json  
  inflating: __MACOSX/proj1/proj1/._result.json  
   creating: proj1/proj1/images/
  inflating: __MACOSX/proj1/proj1/._images  
  inflating: proj1/proj1/images/4d541ded-1uzz3j_imgur_NVDB9Yw.jpg  
  inflating: __MACOSX/proj1/proj1/images/._4d541ded-1uzz3j_imgur_NVDB9Yw.jpg  
  inflating: proj1/proj1/images/d0f632fd-1vfxw0.jpg  

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import cv2
import numpy as np

In [3]:
# Define the object detection model architecture
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [4]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
# Load the training and validation datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'proj1/',
    labels='inferred',
    # label_mode='binary',
    batch_size=32,
    image_size=(224, 224))

# val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     'val/',
#     labels='inferred',
#     label_mode='binary',
#     batch_size=32,
#     image_size=(224, 224))

Found 7 files belonging to 1 classes.


In [8]:
# Preprocess the images
def preprocess_image(image):
    image = tf.image.resize(image, (224, 224))
    image = tf.cast(image, tf.float32) / 255.0
    return image

train_dataset = train_dataset.map(lambda x, y: (preprocess_image(x), y))
# val_dataset = val_dataset.map(lambda x, y: (preprocess_image(x), y))

In [9]:
# Train the model
model.fit(train_dataset,
          # validation_data=val_dataset,
          epochs=10)

# Save the model
model.save('watermark_detection_model.h5')

Epoch 1/10
1/1 [==============================] - 4s 4s/step - loss: 0.6932 - accuracy: 0.4286
Epoch 2/10
1/1 [==============================] - 1s 718ms/step - loss: 0.6273 - accuracy: 1.0000
Epoch 3/10
1/1 [==============================] - 1s 759ms/step - loss: 0.3617 - accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 1s 745ms/step - loss: 0.0781 - accuracy: 1.0000
Epoch 5/10
1/1 [==============================] - 1s 1s/step - loss: 0.0030 - accuracy: 1.0000
Epoch 6/10
1/1 [==============================] - 1s 1s/step - loss: 2.7953e-05 - accuracy: 1.0000
Epoch 7/10
1/1 [==============================] - 1s 815ms/step - loss: 7.9373e-08 - accuracy: 1.0000
Epoch 8/10
1/1 [==============================] - 1s 709ms/step - loss: 8.5528e-11 - accuracy: 1.0000
Epoch 9/10
1/1 [==============================] - 1s 734ms/step - loss: 4.2961e-14 - accuracy: 1.0000
Epoch 10/10
1/1 [==============================] - 1s 697ms/step - loss: 1.2216e-17 - accuracy: 1.0000


In [10]:
# Load the test image
image = cv2.imread('watermark1/10002.jpg')

# Preprocess the test image
image = cv2.resize(image, (224, 224))
image = np.expand_dims(image, axis=0)
image = image.astype('float32') / 255.0

# Use the model to predict the bounding box around the watermark
predictions = model.predict(image)
if predictions[0][0] > 0.5:
    print('Watermark detected')
    cv2.rectangle(image, (0, 0), (224, 224), (0, 224, 0), 2)

1/1 [==============================] - 0s 231ms/step
